In [1]:
import polars as pl
import pandas as pd
import numpy as np

In [6]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

In [7]:
# normalize data
players = players.with_columns([pl.col('nflId').cast(str)])
plays = plays.join(games,on='gameId')
plays = plays.with_columns([
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId')
])

tracking = tracking.with_columns(
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId'),
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str) + '-'
     + pl.col('nflId').cast(str)).alias('uniquePlayerId'),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(53.3-pl.col('y')).otherwise(pl.col('y')).alias('adjustedX'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('x')).otherwise(120-pl.col('x')).alias('adjustedY')
])

tracking=tracking.with_columns([
    pl.when(pl.col('event')=='ball_snap').then(pl.col('frameId')).otherwise(-1).alias('startingFrameId'),
])
tracking=tracking.with_columns([
    pl.col('startingFrameId').max().over(pl.col('uniquePlayId')),
])
tracking=tracking.with_columns([
    (pl.col('frameId') - pl.col('startingFrameId')).alias('framesSinceSnap'),
])

In [16]:
test = tracking.filter(tracking['playId']==56)
test.filter(test['framesSinceSnap']==2)

gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,uniquePlayId,uniquePlayerId,adjustedX,adjustedY,startingFrameId,framesSinceSnap
i64,i64,str,str,i64,str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,f64,f64,i64,i64
2022090800,56,"""35472""","""Rodger Saffold…",1,"""2022-09-08 20:…","""76""","""BUF""","""left""",88.37,27.27,1.62,1.15,0.16,"""231.74""","""147.9""","""NA""","""2022090800-56""","""2022090800-56-…",27.27,31.63,-1,2
2022090800,56,"""38577""","""Bobby Wagner""",1,"""2022-09-08 20:…","""45""","""LA""","""left""",78.25,27.53,1.24,3.81,0.11,"""4.2""","""331.67""","""NA""","""2022090800-56""","""2022090800-56-…",27.53,41.75,-1,2
2022090800,56,"""41239""","""Aaron Donald""",1,"""2022-09-08 20:…","""99""","""LA""","""left""",91.08,31.61,3.91,2.34,0.4,"""168.33""","""132.02""","""NA""","""2022090800-56""","""2022090800-56-…",31.61,28.92,-1,2
2022090800,56,"""42392""","""Mitch Morse""",1,"""2022-09-08 20:…","""60""","""BUF""","""left""",88.25,28.74,0.67,1.97,0.07,"""244.21""","""30.29""","""NA""","""2022090800-56""","""2022090800-56-…",28.74,31.75,-1,2
2022090800,56,"""42489""","""Stefon Diggs""",1,"""2022-09-08 20:…","""14""","""BUF""","""left""",80.6,38.29,6.23,0.9,0.62,"""140.05""","""189.38""","""NA""","""2022090800-56""","""2022090800-56-…",38.29,39.4,-1,2
2022090800,56,"""42816""","""Troy Hill""",1,"""2022-09-08 20:…","""2""","""LA""","""left""",72.34,7.81,4.36,0.91,0.43,"""338.33""","""257.23""","""NA""","""2022090800-56""","""2022090800-56-…",7.81,47.66,-1,2
2022090800,56,"""43294""","""Jalen Ramsey""",1,"""2022-09-08 20:…","""5""","""LA""","""left""",77.95,40.78,5.5,3.0,0.54,"""139.54""","""172.5""","""NA""","""2022090800-56""","""2022090800-56-…",40.78,42.05,-1,2
2022090800,56,"""43298""","""Leonard Floyd""",1,"""2022-09-08 20:…","""54""","""LA""","""left""",91.99,33.78,1.86,4.76,0.2,"""155.94""","""136.01""","""NA""","""2022090800-56""","""2022090800-56-…",33.78,28.01,-1,2
2022090800,56,"""43335""","""A'Shawn Robins…",1,"""2022-09-08 20:…","""94""","""LA""","""left""",88.51,25.47,1.09,2.88,0.1,"""13.98""","""43.43""","""NA""","""2022090800-56""","""2022090800-56-…",25.47,31.49,-1,2


In [13]:
def is_in_vision_cone(index, df):
    # Calculate angle between origin and point
    angle = math.atan2(y1 - y_origin, x1 - x_origin)
    # Convert angle to degrees
    angle_degrees = math.degrees(angle) % 360
    # Calculate angle difference between the point and the cone's direction
    angle_difference = abs((angle_degrees - degree + 360) % 360 - 180)
    
    # Check if the point is within the cone's vision
    if angle_difference <= 0.5 * degree and math.sqrt((x1 - x_origin) ** 2 + (y1 - y_origin) ** 2) <= radius:
        return True
    else:
        return False

IndentationError: expected an indented block (259549654.py, line 1)